# Trigger Trading Strategy

### description



In [1]:
# packages
import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf

import utils

In [2]:
def get_data(ticker: str) -> pd.DataFrame:

    INTERVALS = ["1m", "2m", "5m", "15m", "30m"]
    intervals = utils.fetch_intervals()

    pxs = []
    for interval, from_date, to_date  in intervals:
        
        if interval not in INTERVALS:
            continue

        px = yf.download(ticker, start=from_date, end=to_date, interval=interval, progress=False)
        px["interval"] = interval
        pxs.append(px)

    px = pd.concat(pxs, axis=0)
    px = px.sort_index()

    px = px["Adj Close"].to_frame(name=ticker)
    
    return px



In [44]:
# settings

TRIGGER_ASSETS = ["GBPUSD=X", "CL=F"]
TRADE_ASSETS = ["^FTSE", "EUEA.AS"]

ASSETS = TRIGGER_ASSETS + TRADE_ASSETS

FORWARD_WINDOW = 30
SEQUENCE_LENGTH = 60

## Get Data

In [45]:
df = pd.concat([get_data(ticker=ticker) for ticker in ASSETS], axis=1)
df["date"] = df.index.date
df = df.groupby("date").ffill()
df = df.dropna()
df["date"] = df.index.date
ret = df.groupby("date").pct_change().dropna()
ret.head(10)

,GBPUSD=X,CL=F,^FTSE,EUEA.AS
Datetime,,,,
2024-01-29 08:06:00+00:00,0.000114,0.000765,0.000394,-0.000530
2024-01-29 08:08:00+00:00,-0.000114,-0.000764,0.000385,0.000212
2024-01-29 08:10:00+00:00,0.000127,-0.000765,0.000101,0.000000
2024-01-29 08:12:00+00:00,-0.000051,0.000638,-0.000653,0.000424
2024-01-29 08:14:00+00:00,-0.000038,-0.000510,-0.000296,-0.000212
2024-01-29 08:16:00+00:00,-0.000394,0.000638,-0.000123,-0.000318
2024-01-29 08:18:00+00:00,0.000064,0.000255,0.000271,0.000425
2024-01-29 08:20:00+00:00,0.000165,-0.000127,-0.000122,-0.000318
2024-01-29 08:22:00+00:00,-0.000076,-0.000255,0.000497,0.000000


In [46]:
X_list = []
y_list = []

for i in range(SEQUENCE_LENGTH, ret.shape[0] - FORWARD_WINDOW):
    timestamp = ret.index[i]
    from_timestamp = ret.index[i - SEQUENCE_LENGTH + 1]
    to_timestamp = ret.index[i + FORWARD_WINDOW]

    if from_timestamp.date() != to_timestamp.date():
        continue

    before_data = ret.loc[(ret.index >= from_timestamp) & (ret.index <= timestamp)]
    after_data = ret.loc[(ret.index > timestamp) & (ret.index <= to_timestamp)]

    y = ((after_data["EUEA.AS"] - after_data["^FTSE"]) + 1).prod() - 1
    X = before_data.T.values

    y_list.append(y)
    X_list.append(X)

y = np.array(y_list)
X = np.array(X_list)

In [47]:
X.shape

(18800, 4, 60)

In [48]:
y.shape

(18800,)

## Model

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [50]:
from loguru import logger
import torch


def get_device() -> torch.device:

    if torch.cuda.is_available():
        logger.info("Running on the GPU")
        n_gpu = 0 # different if you have more than 1
        assert n_gpu <= (torch.cuda.device_count() - 1)
        logger.info(f"Using gpu={n_gpu} out of {torch.cuda.device_count()}")
        device = torch.device(f"cuda:{n_gpu}")
    else:
        logger.info("Running on the CPU")
        device = torch.device("cpu")
    
    return device


class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=2, kernel_size=2)
        
        self.fc1 = nn.Linear(in_features=112, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

DEVICE = get_device()

2024-03-03 22:15:29.630 | INFO     | __main__:get_device:14 - Running on the CPU


In [51]:
net = Net().to(device=DEVICE)

In [52]:
# SAMPLE_SIZE = 128

# X_tensor = torch.tensor(X[:SAMPLE_SIZE], dtype=torch.float32)
# X_tensor = X_tensor.reshape(SAMPLE_SIZE, 1, X_tensor.shape[1], X_tensor.shape[2])
# y_tensor = torch.tensor(y[:SAMPLE_SIZE], dtype=torch.float32)

X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor = X_tensor.reshape(X_tensor.shape[0], 1, X_tensor.shape[1], X_tensor.shape[2])
y_tensor = torch.tensor(y, dtype=torch.float32)

In [53]:
print(f"{X_tensor.shape = }")
print(f"{y_tensor.shape = }")

X_tensor.shape = torch.Size([18800, 1, 4, 60])
y_tensor.shape = torch.Size([18800])


In [54]:
from sklearn.model_selection import train_test_split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, train_size=0.7, shuffle=False)

In [56]:
print(f"{X_train.shape = }")
print(f"{X_test.shape = }")
print(f"{y_train.shape = }")
print(f"{y_test.shape = }")

X_train.shape = torch.Size([13160, 1, 4, 60])
X_test.shape = torch.Size([5640, 1, 4, 60])
y_train.shape = torch.Size([13160])
y_test.shape = torch.Size([5640])


In [57]:
# Set Loss function with criterion
criterion = nn.MSELoss()
# Set optimizer with optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)  

In [60]:
EPOCHS = 25
BATCH_SIZE = 128

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(EPOCHS):

    net.train()
    
    for i in range(0, X_train.shape[0] - BATCH_SIZE, BATCH_SIZE):
            
        # Move tensors to the configured device
        batch_X = X_train[i:(i+BATCH_SIZE)].to(device=DEVICE)
        batch_y = y_train[i:(i+BATCH_SIZE)].to(device=DEVICE)

        if batch_X.shape != BATCH_SIZE:
            continue
        
        # Forward pass
        y_hat = net(batch_X)
        loss = criterion(y_hat, batch_y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("hi")
    net.eval()
    y_test_hat = net(X_test)
    loss_test = criterion(y_test_hat, y_test)

    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss}, Val Loss: {loss_test}")


hi


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5640x232 and 112x32)

In [ ]:
y_train_hat = net(X_train)

In [ ]:
y_train_hat[y_train_hat > 1 / 100_000]

In [62]:
net(X_test)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5640x232 and 112x32)